In [2]:
import numpy as np # 导入packages
import time

In [3]:
# 读取制作的fq文件
with open("D:/华为SG/Ori_ACO_DATA.fq"   , 'rt') as f: #"D:/华为SG/Ori_ACO_DATA.fq"   
    text = f.readlines()
    qual_list = []
    for i in range(len(text)//4):
        qual_list.append(text[4*i+3])
    base_list = []
    for j in range(len(text)//4):
        base_list.append(text[4*j+1][:150]) #对于华大的数据集，第151列是无效数据，所以此处只取前150
        #若是illumina的数据集，其列数为101/103交替， 只取前100列

In [4]:
# Quality Scores 编码为uint
ascii_list = list()

for i in range(len(qual_list)):
    trial = list(str.encode(qual_list[i]))
    trial_np = np.array(trial, dtype=np.uint8)
    ascii_list.append(trial_np)

In [5]:
ascii_list_100 = []

for i in range(len(ascii_list)):
    ascii_list_100.append(ascii_list[i][:150])  ##########

In [6]:
# 转化为矩阵
matrix = np.array(ascii_list_100)

In [7]:
matrix

array([[70, 70, 70, ..., 70, 71, 70],
       [70, 70, 70, ..., 70, 68, 67],
       [70, 61, 69, ..., 71, 68, 69],
       ...,
       [69, 70, 70, ..., 70, 70, 66],
       [53, 51, 56, ..., 46, 63, 68],
       [68, 66, 69, ..., 69, 68, 68]], dtype=uint8)

In [8]:
matrix.shape

(124993, 150)

In [9]:
# 修改参数
matrix = matrix[:60000,:]
base_list = base_list[:60000]

In [10]:
matrix.shape

(60000, 150)

In [11]:
len(base_list[1])

150

In [12]:
# 分段1
def quan(q):
    if q < 60:
        q = 60
    elif 60 <= q < 62:
        q = 62
    elif 62 <= q < 64:
        q = 64
    elif 64 <= q < 66:
        q = 66
    elif 66 <= q < 68:
        q = 68
    elif 68 <= q < 70:
        q = 70
    else:
        q = q
    return q


In [40]:
# 分段2
# 
# quan_0 = round(np.percentile(row_mean,0,interpolation='midpoint'))
# quan_25 = round(np.percentile(row_mean,25,interpolation='midpoint'))
# quan_50 = round(np.percentile(row_mean,50,interpolation='midpoint'))
# quan_75 = round(np.percentile(row_mean,75,interpolation='midpoint'))
# quan_100 = round(np.percentile(row_mean,100,interpolation='midpoint'))
# print(quan_0, quan_25, quan_50, quan_75, quan_100)

# def quan(q):
#     if q < quan_0:
#         q = quan_0
#     elif quan_0 <= q < quan_25:
#         q = quan_25
#     elif quan_25 <= q < quan_50:
#         q = quan_50
#     elif quan_50 <= q < quan_75:
#         q = quan_75
#     elif quan_75 <= q < quan_100:
#         q = quan_100
#     else:
#         q = q
#     return q

54.0 67.0 69.0 69.0 70.0


In [34]:
# 分段3
# def quan(q):
#     if q < 60:
#         q = 60
#     elif 60 <= q < 62:
#         q = 62
#     elif 62 <= q < 64:
#         q = 64
#     elif 64 <= q < 66:
#         q = 66
#     elif 66 <= q < 68:
#         q = 68
#     elif 68 <= q:
#         q = 70
#     else:
#         q = q
#     return q

In [60]:
# 分段4
# row_mean = matrix.mean(axis = 1)
# quan_0 = round(np.percentile(row_mean,0,interpolation='midpoint'))
# quan_125 = round(np.percentile(row_mean,12.5,interpolation='midpoint'))
# quan_250 = round(np.percentile(row_mean,25,interpolation='midpoint'))
# quan_375 = round(np.percentile(row_mean,37.5,interpolation='midpoint'))
# quan_500 = round(np.percentile(row_mean,50,interpolation='midpoint'))
# quan_625 = round(np.percentile(row_mean,62.5,interpolation='midpoint'))
# quan_750 = round(np.percentile(row_mean,75,interpolation='midpoint'))
# quan_875 = round(np.percentile(row_mean,87.5,interpolation='midpoint'))
# quan_1000 = round(np.percentile(row_mean,100,interpolation='midpoint'))
# print(quan_0, quan_125, quan_250, quan_375, quan_500, quan_625, quan_750, quan_875, quan_1000)

# def quan(q):
#     if q < quan_0:
#         q = quan_0
#     elif quan_0 <= q < quan_125:
#         q = quan_125
#     elif quan_125 <= q < quan_250:
#         q = quan_250
#     elif quan_250 <= q < quan_375:
#         q = quan_375
#     elif quan_375 <= q < quan_500:
#         q = quan_500
#     elif quan_500 <= q < quan_625:
#         q = quan_625
#     elif quan_625 <= q < quan_750:
#         q = quan_750
#     elif quan_750 <= q < quan_875:
#         q = quan_875
#     elif quan_875 <= q:
#         q = quan_1000
#     else:
#         q = q
#     return q

In [13]:
# 概率表
row_mean = matrix.mean(axis = 1)
total_entropy = 0
freq_dic = {}
starttime = time.time()

for k in range(len(row_mean)):
    row_mean[k] = quan(row_mean[k])

for j in range(4,matrix.shape[1]):#150
    for i in range(2, matrix.shape[0]):#10000
        #context_list = (max(matrix[i,j-4],matrix[i,j-3]), max(matrix[i,j-2],matrix[i,j-1]), min(matrix[i-2,j],matrix[i-1,j]),row_mean[i])
        context_list = (#max(matrix[i,j-4],matrix[i,j-3]),
                        max(matrix[i,j-2],matrix[i,j-1]), 
                         row_mean[i])    #base_list[i][j], base_list[i][j-1],
        #context_list = (max(matrix[i,j-2],matrix[i,j-1]),row_mean[i])
        if context_list not in freq_dic.keys():
            freq_dic[context_list] = {matrix[i,j]:1}
            total_entropy += 8
        else:
            if matrix[i,j] not in freq_dic[context_list].keys():
                freq_dic[context_list].update({matrix[i,j]:1})
                total_entropy += 8
            else:
                total_entropy += -np.log2(freq_dic[context_list][matrix[i,j]]/np.sum(list(freq_dic[context_list].values())))
                freq_dic[context_list][matrix[i,j]] += 1

endtime = time.time()
print("total_time: ",(endtime - starttime))

total_time:  129.67201399803162


In [14]:
total_entropy

24196006.47324472

In [109]:
freq_dic

{(70, 70.0): {35: 1, 50: 1, 70: 3, 68: 1, 69: 1},
 (50, 70.0): {65: 1},
 (65, 70.0): {68: 1},
 (68, 70.0): {72: 1, 68: 20, 63: 1, 66: 3, 62: 1},
 (72, 70.0): {70: 3, 72: 7, 74: 1},
 (74, 70.0): {73: 5, 74: 30, 72: 3},
 (73, 70.0): {73: 2, 74: 2},
 (69, 70.0): {69: 2, 68: 2},
 (66, 70.0): {68: 1, 67: 1},
 (67, 70.0): {67: 1, 68: 1}}

In [110]:
matrix[2]

array([67, 67, 67, 70, 35, 50, 65, 68, 72, 70, 72, 72, 72, 74, 73, 74, 74,
       74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 73,
       74, 74, 74, 74, 72, 73, 73, 74, 74, 74, 74, 74, 74, 74, 74, 73, 73,
       73, 74, 74, 72, 72, 72, 72, 72, 72, 70, 70, 70, 70, 70, 68, 69, 69,
       69, 68, 68, 68, 68, 68, 68, 63, 66, 68, 68, 68, 68, 68, 68, 68, 68,
       68, 66, 68, 68, 68, 68, 68, 68, 68, 66, 62, 67, 67, 68, 68],
      dtype=uint8)

1819087.7114509365

In [18]:
freq_dic

{(67, 70, 'C', 'A', 70.18): {35: 1, 70: 6},
 (51, 61, 'C', 'A', 68.0): {68: 1},
 (67, 70, 'C', 'A', 70.0): {35: 8,
  70: 186,
  68: 20,
  65: 1,
  72: 1,
  67: 2,
  66: 2,
  69: 1,
  54: 1},
 (68, 67, 'C', 'A', 72.34): {70: 4},
 (64, 70, 'C', 'A', 70.09): {35: 1},
 (63, 61, 'C', 'A', 70.55): {66: 1},
 (64, 68, 'C', 'A', 70.0): {35: 1,
  68: 59,
  70: 20,
  49: 1,
  66: 8,
  64: 1,
  63: 2,
  71: 2,
  67: 13,
  65: 3,
  69: 2,
  58: 1,
  51: 1},
 (64, 67, 'C', 'A', 70.5): {70: 1},
 (67, 70, 'C', 'A', 68.0): {35: 2,
  70: 47,
  63: 1,
  68: 6,
  65: 2,
  73: 1,
  72: 2,
  69: 2,
  64: 1,
  66: 1,
  71: 1,
  67: 2},
 (60, 61, 'C', 'A', 68.0): {68: 1},
 (67, 70, 'C', 'A', 71.02): {35: 1, 70: 6},
 (67, 61, 'C', 'A', 70.58): {68: 1},
 (63, 66, 'C', 'A', 66.0): {35: 1,
  68: 5,
  63: 1,
  57: 1,
  67: 1,
  66: 2,
  60: 1,
  53: 1,
  56: 1,
  64: 1,
  58: 1},
 (65, 66, 'C', 'A', 64.0): {68: 5, 66: 2, 57: 1, 65: 1, 59: 1},
 (67, 70, 'C', 'A', 71.38): {35: 1, 70: 5},
 (64, 61, 'C', 'A', 71.92): 